# Semantic Kernel עם אינטגרציה של שרת OpenBnB MCP

מחברת זו מדגימה כיצד להשתמש ב-Semantic Kernel יחד עם שרת OpenBnB MCP האמיתי כדי לחפש לינות אמיתיות ב-Airbnb באמצעות MCPStdioPlugin. לגישה ל-LLM, היא משתמשת ב-Microsoft Foundry. כדי להגדיר את משתני הסביבה שלך, תוכל לעקוב אחרי [שיעור ההגדרה](/00-course-setup/README.md)


## ייבוא החבילות הנדרשות


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## יצירת חיבור התוסף MCP

נחבר ל[שרת MCP של OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) באמצעות MCPStdioPlugin. שרת זה מספק פונקציונליות חיפוש של Airbnb דרך החבילה @openbnb/mcp-server-airbnb.


## יצירת הלקוח

בדוגמה זו, נשתמש ב-Microsoft Foundry כדי לגשת למודל השפה הגדול שלנו. ודא שמשתני הסביבה שלך מוגדרים כראוי.


## קונפיגורציית סביבה

קבע את הגדרות Azure OpenAI. וודא שיש לך את משתני הסביבה הבאים מוגדרים:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## הבנת אינטגרציית OpenBnB MCP

פנקס העבודות הזה מתחבר אל **שרת OpenBnB MCP האמיתי** שמספק פונקציונליות חיפוש אמיתית של Airbnb.

### איך זה עובד:

1. **MCPStdioPlugin**: משתמש בתקשורת קלט/פלט סטנדרטית עם שרת ה-MCP  
2. **חבילת NPM אמיתית**: מוריד ומריץ את `@openbnb/mcp-server-airbnb` דרך npx  
3. **נתונים חיים**: מחזיר נתוני נכסים אמיתיים של Airbnb מה-APIs שלהם  
4. **גילוי פונקציות**: הסוכן מגלה אוטומטית את הפונקציות הזמינות מהשרת MCP  

### פונקציות זמינות:

שרת OpenBnB MCP בדרך כלל מספק:  
- **search_listings** - חיפוש נכסי Airbnb לפי מיקום וקריטריונים  
- **get_listing_details** - קבלת מידע מפורט על נכסים ספציפיים  
- **check_availability** - בדיקת זמינות לתאריכים ספציפיים  
- **get_reviews** - שליפת ביקורות על נכסים  
- **get_host_info** - קבלת מידע על מארחי הנכסים  

### דרישות מוקדמות:

- **Node.js** מותקן במערכת שלך  
- **חיבור לאינטרנט** להורדת חבילת שרת ה-MCP  
- **NPX** זמין (כלול עם Node.js)  

### בדיקת החיבור:

ניתן לבדוק את שרת ה-MCP ידנית באמצעות הרצה של:  
```bash
npx -y @openbnb/mcp-server-airbnb
```
  
זה יוריד ויקם את שרת OpenBnB MCP, שאליו Semantic Kernel מתחבר לקבלת נתוני Airbnb אמיתיים.


## הפעלת הסוכן עם שרת OpenBnB MCP

כעת נפעיל את סוכן ה-AI שמתחבר לשרתי OpenBnB MCP כדי לחפש מקומות לינה אמיתיים ב-Airbnb בסטוקהולם עבור 2 מבוגרים ו־1 ילד. ניתן לשנות את רשימת `user_inputs` כדי להתאים את קריטריוני החיפוש.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# סיכום
ברכות! בנית בהצלחה סוכן בינה מלאכותית המתממשק עם חיפוש אירוח בעולם האמיתי באמצעות פרוטוקול הקשר דגם (MCP):

## טכנולוגיות שנעשה בהן שימוש:
- Semantic Kernel - לבניית סוכנים אינטליגנטים עם Azure OpenAI
- Microsoft Foundry - ליכולות LLM והשלמת שיחות
- MCP (Model Context Protocol) - לאינטגרציה סטנדרטית של כלים
- שרת MCP של OpenBnB - לפונקציונליות חיפוש Airbnb אמיתית
- Node.js/NPX - להרצת שרת MCP חיצוני

## מה שלמדת:
- אינטגרציית MCP: חיבור סוכני Semantic Kernel לשרתים חיצוניים של MCP
- גישה בזמן אמת לנתונים: חיפוש נכסי Airbnb אמיתיים דרך APIs חיים
- תקשורת פרוטוקול: שימוש בתקשורת stdio בין הסוכן לשרת MCP
- גילוי פונקציות: גילוי אוטומטי של פונקציות זמינות משרתים של MCP
- זרימת תגובות: לכידה ורישום של קריאות פונקציות בזמן אמת
- עיצוב HTML: עיצוב תגובות סוכן בטבלאות מעוצבות ותצוגות אינטראקטיביות

## השלבים הבאים:
- אינטגרציה של שרתי MCP נוספים (מזג אוויר, טיסות, מסעדות)
- בניית מערכת רב-סוכנים המשלבת פרוטוקולי MCP ו-A2A
- יצירת שרתי MCP מותאמים אישית למקורות הנתונים שלך
- יישום זיכרון שיחה מתמשך בין הפעלות
- פריסת הסוכן ל-Azure Functions עם אורקסטרציה של שרת MCP
- הוספת אימות משתמש ויכולות הזמנה


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**כתב ויתור**:
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). אמנם אנו שואפים לדייק, יש לקחת בחשבון כי תרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפת המקור שלו נחשב למקור הסמכותי. למידע קריטי מומלץ להיעזר בתרגום מקצועי על ידי אדם. אנו לא נושאים באחריות לכל אי הבנה או פרשנות שגויה הנובעת משימוש בתרגום זה.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
